In [1]:
import sys

sys.path.insert(0, '..')

import shutil
import os
import uuid
from biosimulator_processes.execute import exec_utc_comparison
from verification_service.worker.compare import Supervisor, Worker
from verification_service.data_model.shared import MongoDbConnector
from pymongo.mongo_client import MongoClient
from dotenv import load_dotenv
import os 

_outs = './test_outputs'
if os.path.exists(_outs):
    shutil.rmtree(_outs)


def jobid(): return str(uuid.uuid4())


load_dotenv("../verification_service/.env")
uri = os.getenv("MONGO_DB_URI")
omex_source_dir = './examples/sbml-core'
omex_name = 'Elowitz-Nature-2000-Repressilator.omex'
omex_fp = os.path.join(omex_source_dir, omex_name)
out_dir = './test_outputs'
simulators = ['amici', 'copasi', 'tellurium']
spec_name = 'cI mRNA'
job_id = jobid()
db_connector = MongoDbConnector(client=MongoClient(uri), database_id="service_requests")

Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.
Cannot register SimpleNeuron. Error:
**
No module named 'pyneuroml'
**


In [2]:
supervisor = Supervisor(db_connector=db_connector)

In [3]:
supervisor.jobs

{'completed_jobs': [],
 'in_progress_jobs': [],
 'pending_jobs': ['33a977f1-595a-47ff-aa3e-e132cd629b87',
  'e097bfdf-ee21-4d6e-baea-28829f3140b5']}

In [4]:
supervisor_response = supervisor.initialize()

In [5]:
supervisor_response

In [6]:
supervisor_response

In [7]:
supervisor.jobs

{'completed_jobs': [],
 'in_progress_jobs': [],
 'pending_jobs': ['33a977f1-595a-47ff-aa3e-e132cd629b87',
  'e097bfdf-ee21-4d6e-baea-28829f3140b5']}

In [8]:
[v for v in supervisor.db_connector.db.completed_jobs.find()]

[]

In [4]:
job_id = supervisor.jobs['pending_jobs'][0]

In [5]:
job_id

'33a977f1-595a-47ff-aa3e-e132cd629b87'

In [6]:
job_doc = supervisor.db_connector.db.pending_jobs.find_one({'job_id': job_id})

In [7]:
job_doc

{'_id': ObjectId('6677a7117bb69cccd7ac2c0f'),
 'status': 'PENDING',
 'job_id': '33a977f1-595a-47ff-aa3e-e132cd629b87',
 'omex_path': './tmp/Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations.omex',
 'simulators': ['amici', 'copasi', 'tellurium'],
 'comparison_id': 'test-0',
 'timestamp': '2024-06-23 04:39:45.133127',
 'reports_path': None,
 'include_output': True}

In [8]:
job_params = job_doc.copy()

In [9]:
list(map(lambda k: job_params.pop(k), ['job_id', 'status', 'timestamp', '_id']))

['33a977f1-595a-47ff-aa3e-e132cd629b87',
 'PENDING',
 '2024-06-23 04:39:45.133127',
 ObjectId('6677a7117bb69cccd7ac2c0f')]

In [10]:
job_params

{'omex_path': './tmp/Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations.omex',
 'simulators': ['amici', 'copasi', 'tellurium'],
 'comparison_id': 'test-0',
 'reports_path': None,
 'include_output': True}

In [11]:
worker = Worker(job_params=job_params)

TypeError: utc_comparison() got an unexpected keyword argument 'reports_path'